In [1]:
import caffe
import os

In [2]:
num_points = 8

model_stage1 = '../models/FLD_full/stage1.prototxt'
assert(os.path.exists(model_stage1))
weights_stage1 = '../models/FLD_full/FLD_full_models/stage1.caffemodel'
assert(os.path.exists(weights_stage1))

model_stage2 = '../models/FLD_full/cascade.prototxt'
assert(os.path.exists(model_stage2))
weights_stage2 = '../models/FLD_full/FLD_full_models/stage2.caffemodel'
assert(os.path.exists(weights_stage2))

model_stage3 = '../models/FLD_full/cascade.prototxt'
assert(os.path.exists(model_stage3))
weights_stage3_easy = '../models/FLD_full/FLD_full_models/stage3_easy.caffemodel'
assert(os.path.exists(weights_stage3_easy))
weights_stage3_hard = '../models/FLD_full/FLD_full_models/stage3_hard.caffemodel'
assert(os.path.exists(weights_stage3_hard))

In [3]:
# caffe prepare
caffe.set_mode_gpu()
caffe.set_device(0)

In [4]:
# create net and load weights
net_stage1 = caffe.Net(model_stage1, weights_stage1, caffe.TEST); 
net_stage2 = caffe.Net(model_stage2, weights_stage2, caffe.TEST);
net_stage3_easy = caffe.Net(model_stage3, weights_stage3_easy, caffe.TEST);
net_stage3_hard = caffe.Net(model_stage3, weights_stage3_hard, caffe.TEST);
pipeline = {
    'num_points': num_points,
    'net_stage1': net_stage1,
    'net_stage2': net_stage2,
    'net_stage3_easy': net_stage3_easy,
    'net_stage3_hard': net_stage3_hard
}

In [5]:
image_path = '../data/FLD_full/'

In [ ]:
# network processing
# http://caffe.berkeleyvision.org/tutorial/interfaces.html
# http://christopher5106.github.io/deep/learning/2015/09/04/Deep-learning-tutorial-on-Caffe-Technology.html

In [ ]:
# release resource
caffe